In [1]:
import pandas as pd
import requests
import openai
import os
import sys
import plotly.express as px
import json
from typing import List
from pydantic import BaseModel, Field
sys.path.append("../")
from dotenv import load_dotenv
from finance_functions import STOCK, get_ticker

In [2]:
# Load the environment variables from .env file
load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [4]:
import langchain
print(langchain.__version__)

0.3.24


In [5]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI(model="gpt-4.1-nano-2025-04-14")
output_parser = StrOutputParser()

In [6]:
chain = prompt | model | output_parser

In [7]:
chain.invoke({"topic": "bears"})

'Why do bears have sticky paws?  \nBecause they always get into bear-ly any trouble!'

In [8]:
from langchain_core.utils.function_calling import convert_to_openai_function

In [9]:
class GetTicker(BaseModel):
    """Call this with a company name to get the ticker on yahoo finance"""
    company_name: str = Field(description="Company name")

In [10]:
get_ticker_function = convert_to_openai_function(GetTicker) # type: ignore

In [11]:
get_ticker_function

{'name': 'GetTicker',
 'description': 'Call this with a company name to get the ticker on yahoo finance',
 'parameters': {'properties': {'company_name': {'description': 'Company name',
    'type': 'string'}},
  'required': ['company_name'],
  'type': 'object'}}

In [12]:
model_with_function = model.bind(functions=[get_ticker_function])

In [13]:
model_with_function.invoke("What is the ticker for Unicredit?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"company_name":"Unicredit"}', 'name': 'GetTicker'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 65, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45', 'id': 'chatcmpl-BQVCL4iHp8IADXFYeSC2ah09i3pxC', 'finish_reason': 'function_call', 'logprobs': None}, id='run-e3a9eef0-cdfe-4ca0-9822-c50c78048829-0', usage_metadata={'input_tokens': 65, 'output_tokens': 18, 'total_tokens': 83, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [14]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

chain = prompt | model_with_function

In [15]:
chain.invoke({"input": "What is the ticker for Unicredit?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"company_name":"Unicredit"}', 'name': 'GetTicker'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 71, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45', 'id': 'chatcmpl-BQVCMxdv5QErh9aplKBXXl7TuHlc0', 'finish_reason': 'function_call', 'logprobs': None}, id='run-c8ee3326-ace9-4a8c-8e80-08dcb2186b2e-0', usage_metadata={'input_tokens': 71, 'output_tokens': 18, 'total_tokens': 89, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [20]:
chain = prompt | model_with_function

In [22]:
res = chain.invoke({"input": "What is the ticker for Unicredit?"})

In [27]:
res.additional_kwargs

{'function_call': {'arguments': '{"company_name":"Unicredit"}',
  'name': 'GetTicker'},
 'refusal': None}

In [30]:
from langchain.output_parsers.openai_functions import PydanticOutputFunctionsParser
parser = PydanticOutputFunctionsParser(pydantic_schema=GetTicker)

In [31]:
chain = prompt | model_with_function | parser

In [36]:
res = chain.invoke({"input": "What is the ticker for Unicredit?"})

In [37]:
res

GetTicker(company_name='Unicredit')

In [38]:
from langchain_core.tools import tool

In [82]:
@tool
def get_ticker(company_name: str) -> str:
    """
    Get the ticker based on the company name
    Parameters:
    company_name: str
        Free text
    """
    yfinance_url = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = requests.get(url=yfinance_url, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    company_code = data['quotes'][0]['symbol']
    return company_code

In [74]:
get_ticker.invoke("Unicredit")

'UCG.MI'

In [75]:
model_with_tools = model.bind(tools=[get_ticker])

In [76]:
chain = prompt | model_with_tools

In [77]:
from langchain_core.messages import HumanMessage

In [78]:
query = "What is the ticker for Unicredit?"

messages = [HumanMessage(query)]

In [84]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4.1-nano-2025-04-14", model_provider="openai")

In [85]:
tools = [get_ticker]

In [86]:
llm_with_tools = llm.bind_tools(tools)

In [87]:
ai_msg = llm_with_tools.invoke(messages)

In [88]:
print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'get_ticker', 'args': {'company_name': 'Unicredit'}, 'id': 'call_4HOoxDdo0RVDWOKFJW6Fugzu', 'type': 'tool_call'}]


In [89]:
messages

[HumanMessage(content='What is the ticker for Unicredit?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4HOoxDdo0RVDWOKFJW6Fugzu', 'function': {'arguments': '{"company_name":"Unicredit"}', 'name': 'get_ticker'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 69, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45', 'id': 'chatcmpl-BQWLQ7AtG8sVvD6pIh8OX3q47JlGG', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-020e6ebc-c016-42e1-8990-7c0d2da7968e-0', tool_calls=[{'name': 'get_ticker', 'args': {'company_name': 'Unicredit'}, 'id': 'call_4HOoxDdo0RVDWOKFJW6Fugzu', 'type': 'tool_call'}], usage_me

In [90]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_ticker": get_ticker}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is the ticker for Unicredit?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4HOoxDdo0RVDWOKFJW6Fugzu', 'function': {'arguments': '{"company_name":"Unicredit"}', 'name': 'get_ticker'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 69, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45', 'id': 'chatcmpl-BQWLQ7AtG8sVvD6pIh8OX3q47JlGG', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-020e6ebc-c016-42e1-8990-7c0d2da7968e-0', tool_calls=[{'name': 'get_ticker', 'args': {'company_name': 'Unicredit'}, 'id': 'call_4HOoxDdo0RVDWOKFJW6Fugzu', 'type': 'tool_call'}], usage_me